# Kinetic modeling

One of the main applications of metabolic RNA labeling is to model the kinetics of RNA expression levels [[Narain, Ashwin et al. (2021)]](https://www.cell.com/molecular-cell/fulltext/S1097-2765(21)00496-2). The most natural and widely used model is to describe the change of RNA levels $a(t)$ and time $t$ is by the differential equation:

$$\frac{da}{dt} = \sigma - \delta \cdot a(t)$$

Here $\sigma$ is the net synthesis rate of RNA and $\delta$ is the degradation rate. This differential equation can be solved analytically (see e.g. [Jürges et al. (2018)](https://academic.oup.com/bioinformatics/article/34/13/i218/5045735)). Based on this, there are several ways implemented in GrandPy to estimate both $\sigma$ and $\delta$. Here, we will see how to fit this model using non-linear least squares (NLLS) regression on the estimated new and old RNA counts [[Narain, Ashwin et al. (2021)]](https://www.cell.com/molecular-cell/fulltext/S1097-2765(21)00496-2).

To perform an analysis of the RNA dynamics for data from [Finkel et al. (2021)](https://www.nature.com/articles/s41586-021-03610-3), we first load the GrandPy package and then read the GRAND-SLAM output table directly from zenodo.

In [ ]:
import warnings
import grandpy as gp
from grandpy import plot_pca, plot_gene_progressive_timecourse

warnings.filterwarnings("ignore", category=UserWarning)

sars = gp.read_grand("https://zenodo.org/record/5834034/files/sars.tsv.gz", design=("Condition", "dur.4sU", "Replicate"))

Refer to the [loading data and working with GrandPy objects](../notebook_03_loading_data_and_working_with_grandpy_objects.ipynb) notebook to learn more about how to load data. Note that sample metadata has been automatically extracted from the sample names via the design parameter given to `read_grand()`.

In [ ]:
sars.coldata

By default GRAND-SLAM will report data on all genes (with at least one mapped read) and `read_grand()` will read all these genes from the output:

In [ ]:
print(sars)

Thus, we filter to only include genes that have at least 100 reads in the at least 6 samples:

In [ ]:
sars = sars.filter_genes(min_expression=100, min_columns=6)
print(sars)

The actual data is available in `slots`. `read_grand()` adds the read counts, new to total RNA ratios (NTRs) and information on the NTR posterior distribution (alpha, beta).

As a quick quality check, we can inspect a principal component analysis of all samples involved:

In [ ]:
plot_pca(sars)

The samples are colored according to *Condition* (see above in the Coldata table). *Condition* has a special meaning in GrandPy, not only for `plot_pca()`, but also for other analyses (see below). `condition` can be set conveniently using `with_condition()`. For more information, see the [loading data and working with GrandPy objects](../notebook_03_loading_data_and_working_with_grandpy_objects.ipynb) notebook. For `plot_pca()`, the visual attributes can be adapted via the `aest` parameter:

In [ ]:
plot_pca(sars, aest={"color": "duration.4sU.original", "shape": "Condition"})

There are no obvious problems with the samples, even though the virus infected no4sU sample is the bottom-most whereas the other virus infected samples are ordered along infection time from bottom to top.

For the NLLS approach it is important to normalize the data:

In [ ]:
sars = sars.normalize()
print(sars)

Make sure that the normalization you use here is appropriate. Calling `normalize()` will add a slot which is set to be the default slot.

Before we estimate kinetic parameters globally, we inspect an example:

In [ ]:
plot_gene_progressive_timecourse(sars, "SRSF6")

The curves represent the fitted model for this gene. The kinetic modeling by default makes the assumption of steady state gene expression, i.e. that as much RNA is transcribed per time unit as it is degraded. In mathematical terms, it is

$$\frac{da}{dt} = \sigma - \delta \cdot a(t) \quad \Leftrightarrow \quad a(t) = \frac{\sigma}{\delta}$$

For the virus infected samples (“SARS”), this is not the case. So we specify an additional parameter:

In [ ]:
plot_gene_progressive_timecourse(sars, "SRSF6", steady_state={"Mock": True, "SARS": False})

Note that the fit actually changed. Now we are ready to fit the model for each gene. For that we take a closer look at `fit_kinetics()`.

By default, `fit_kinetics()` will run in parallel if the dataset is large enough. This can be controlled via the `max_processes` and `exact_processes` parameter.

`max_processes` defines the maximum number of worker processes (typically corresponding to CPU cores) that the method can use. `exact_processes` defines, whether to use exactly `max_processes`. If set to `True`, the specified number of processes will be launched, regardless of the workload. If it is set to `False`, the number of processes is adjusted dynamically based on the system's number of cores and the size of the task (This is a rough estimate and might not be optimal, depending on the specific system).

Here we only specify the `steady_state` parameter.

In [ ]:
sars = sars.fit_kinetics(steady_state={"Mock": True, "SARS": False})

`fit_kinetics()` added its result to the GrandPy object.

In [ ]:
print(sars)

Note that there are now two analysis tables. We can get more information using `get_analyses()`:

In [ ]:
sars.get_analyses(description=True)

This tells us that the two analysis tables, each have a Synthesis and a Half-life column. We can retrieve the analysis results using the `get_analysis_table()` method:

In [ ]:
sars.get_analysis_table().head()

See the [working with data matrices and analysis results](../notebook_04_working_with_data_matrices_and_analysis_results.ipynb) notebook for more information on how to retrieve data from the GrandPy object.

We can use the `plot_scatter()` function to compare half-lifes from 'Mock' and 'SARS'.

In [ ]:
gp.plot_scatter(sars, x="Mock_Half-life", y="SARS_Half-life",log=True, diagonal=True)

For more on `plot_scatter()`, see the [plotting notebook](...).`fit_kinetics()` can actually compute much more information. To see this, let's look at the `return_fields` parameter.

In [ ]:
small_sars = sars[0:100] # Only use the first 100 genes for faster computation

small_sars = small_sars.fit_kinetics(steady_state={"Mock": True, "SARS": False}, return_fields=["Synthesis", "Degradation", "Half-life", "log_likelihood", "f0", "total", "conf_lower", "conf_upper", "rmse"], show_progress=False)

print(small_sars.get_analyses(description=True))

We could also extract the residuals from the model fit. For this we go back to the entire dataset. Note that our previous fit result will be overwritten. This could be prevented by setting the `name_prefix` parameter.

In [ ]:
sars = sars.fit_kinetics(steady_state={"Mock": True, "SARS": False}, return_fields="residuals")
print(sars.get_analyses(description=True))

Let's plot the distribution of relative residuals for new RNA.

In [ ]:
df = sars.get_analysis_table(columns=["relative_new"], with_gene_info=False)

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

df.columns = df.columns.str.replace('_residuals_relative_new', '')

df_melted = pd.melt(df, var_name="Sample", value_name="Relative residual")

plt.figure(figsize=(10, 6))
sns.set_theme(style="whitegrid")

sns.boxplot(data=df_melted, x='Sample', y='Relative residual', palette='rainbow')

plt.axhline(0, color='black', linewidth=1)

plt.xticks(rotation=90)

plt.ylim(-1.1, 1.1)

plt.xlabel('Sample', fontsize=14)
plt.ylabel('Relative residual', fontsize=14)

plt.tight_layout()
plt.show()

We see that the residuals from the 1h timepoints are systematically below zero. An example of this is 'SMAD3':

In [ ]:
plot_gene_progressive_timecourse(sars, "SMAD3", steady_state={"Mock": True, "SARS": False})

For both 'Mock' and 'SARS', we see that new RNA from 1h is well below the model fit, i.e. has negative residual. This indicates that the effective labeling time was much lower than the nominal labeling time of 1h for these samples, i.e. that actually, these samples should move to the left. This can be rectified by temporal recalibration:

In [ ]:
# This takes a long time to compute properly, so we just take the 10 top expressed genes here for a rough estimate.
sars = sars.calibrate_effective_labeling_time_kinetic_fit(steady_state={"Mock": True, "SARS": False}, n_top_genes=10)

This adds the column *calibrated_time* to the `coldata`.

In [ ]:
sars.coldata

This recalibrated time can now be used for fitting:

In [ ]:
plot_gene_progressive_timecourse(sars, "SMAD3", steady_state={"Mock": True, "SARS": False}, time="calibrated_time", exact_tics=False)

In [ ]:
sars = sars.fit_kinetics(name_prefix="corrected", time="calibrated_time", steady_state={"Mock": True, "SARS": False}, return_fields="residuals")

This indeed corrected the residuals:

In [ ]:
df = sars.get_analysis_table(columns=["corrected.*relative_new"], with_gene_info=False)

df.columns = df.columns.str.replace('_residuals_relative_new', '')
df.columns = df.columns.str.replace('corrected_', '')

df_melted = pd.melt(df, var_name="Sample", value_name="Relative residual")

plt.figure(figsize=(10, 6))
sns.set_theme(style="whitegrid")

sns.boxplot(data=df_melted, x='Sample', y='Relative residual', palette='rainbow')

plt.axhline(0, color='black', linewidth=1)

plt.xticks(rotation=90)

plt.ylim(-1.1, 1.1)

plt.xlabel('Sample', fontsize=14)
plt.ylabel('Relative residual', fontsize=14)

plt.tight_layout()
plt.show()